## Invoice Processing Workflow with Azure AI and Foundry Agents

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity azure-ai-documentintelligence==1.0.0b4 azure-ai-textanalytics==5.3.0

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.textanalytics import TextAnalyticsClient
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
from azure.ai.projects.models import PromptAgentDefinition, CodeInterpreterTool, CodeInterpreterToolAuto
from azure.core.credentials import AzureKeyCredential
load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
azure_language_endpoint = os.getenv("AZURE_LANGUAGE_ENDPOINT")
azure_language_api_key = os.getenv("AZURE_LANGUAGE_API_KEY")
document_intelligence_endpoint = os.getenv("DOCUMENT_INTELLIGENCE_ENDPOINT")
document_intelligence_api_key = os.getenv("DOCUMENT_INTELLIGENCE_API_KEY")

### Setting Up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating an OpenAI Client

In [ ]:
# creating an openai client first
openai_client = project_client.get_openai_client()

### Creating a Text Analytics Client

In [ ]:
ta_credential = AzureKeyCredential(azure_language_api_key)

text_analytics_client = TextAnalyticsClient(
    endpoint=azure_language_endpoint,
    credential=ta_credential
)

### Creating a Function to Perform Text Redaction (PII Masking)

In [ ]:
def PII_Text_Redaction(input_text: str) -> str:
    documents = [
        input_text
    ]

    response = text_analytics_client.recognize_pii_entities(documents, language="en")
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        redacted_text = doc.redacted_text
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("	Category: {}".format(entity.category))
            print("	Confidence Score: {}".format(entity.confidence_score))
            print("	Offset: {}".format(entity.offset))
            print("	Length: {}".format(entity.length))
        return redacted_text

### Creating a Document Intelligence Client

In [ ]:
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=document_intelligence_endpoint, credential=AzureKeyCredential(document_intelligence_api_key)
)

### Creating the Document Analysis Function

In [ ]:
def analyze_invoice(url: str) -> str:
    # Initialize final response string
    finalResponse = ""

    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-invoice", AnalyzeDocumentRequest(url_source=url)
    )
    invoices = poller.result()
    
    for idx, invoice in enumerate(invoices.documents):
        print("--------Recognizing invoice #{}--------".format(idx + 1))
        vendor_name = invoice.fields.get("VendorName")
        if(vendor_name):
            finalResponse = finalResponse + "vendor name:" + str(vendor_name.get('content')) + "\n"

        vendor_address = invoice.fields.get("VendorAddress")
        if(vendor_address):
            finalResponse = finalResponse + "vendor address:" + str(vendor_address.get('content')) + "\n"
            
        vendor_address_recipient = invoice.fields.get("VendorAddressRecipient")
        if(vendor_address_recipient):
            finalResponse = finalResponse + "vendor address recipient" + str(vendor_address_recipient.get('content')) + "\n"

        customer_name = invoice.fields.get("CustomerName")
        if(customer_name):
            finalResponse = finalResponse + "customer name:" + str(customer_name.get('content')) + "\n"

        customer_id = invoice.fields.get("CustomerId")
        if(customer_id):
            finalResponse = finalResponse + "customer id:" + str(customer_id.get('content')) + "/n"

        customer_address = invoice.fields.get("CustomerAddress")
        if(customer_address):
            finalResponse = finalResponse + "customer address" + str(customer_address.get('content')) + "\n"
            
        customer_address_recipient = invoice.fields.get("CustomerAddressRecipient")
        if(customer_address_recipient):
            finalResponse = finalResponse + "customer address recipient" + str(customer_address_recipient.get('content')) + "\n"

        invoice_id = invoice.fields.get("InvoiceId")
        if(invoice_id):
            finalResponse = finalResponse + "invoice id" + str(invoice_id.get('content')) + "\n"

        invoice_date = invoice.fields.get("InvoiceDate")
        if(invoice_date):
            finalResponse = finalResponse + "invoice date" + str(invoice_date.get('content')) + "\n"

        invoice_total = invoice.fields.get("InvoiceTotal")
        if(invoice_total):
            finalResponse = finalResponse + "invoice total" + str(invoice_total.get('content')) + "\n"

        due_date = invoice.fields.get("DueDate")
        if due_date:
           finalResponse += "due date: " + str(due_date.get('content')) + "\n"

        purchase_order = invoice.fields.get("PurchaseOrder")
        if purchase_order:
           finalResponse += "purchase order: " + str(purchase_order.get('content')) + "\n"

        billing_address = invoice.fields.get("BillingAddress")
        if billing_address:
           finalResponse += "billing address: " + str(billing_address.get('content')) + "\n"

        billing_address_recipient = invoice.fields.get("BillingAddressRecipient")
        if billing_address_recipient:
           finalResponse += "billing address recipient: " + str(billing_address_recipient.get('content')) + "\n"

        shipping_address = invoice.fields.get("ShippingAddress")
        if shipping_address:
           finalResponse += "shipping address: " + str(shipping_address.get('content')) + "\n"

        shipping_address_recipient = invoice.fields.get("ShippingAddressRecipient")
        if shipping_address_recipient:
           finalResponse += "shipping address recipient: " + str(shipping_address_recipient.get('content')) + "\n"

        subtotal = invoice.fields.get("SubTotal")
        if subtotal:
           finalResponse += "subtotal: " + str(subtotal.get('content')) + "\n"

        total_tax = invoice.fields.get("TotalTax")
        if total_tax:
           finalResponse += "total tax: " + str(total_tax.get('content')) + "\n"

        previous_unpaid_balance = invoice.fields.get("PreviousUnpaidBalance")
        if previous_unpaid_balance:
           finalResponse += "previous unpaid balance: " + str(previous_unpaid_balance.get('content')) + "\n"

        amount_due = invoice.fields.get("AmountDue")
        if amount_due:
           finalResponse += "amount due: " + str(amount_due.get('content')) + "\n"

        service_start_date = invoice.fields.get("ServiceStartDate")
        if service_start_date:
           finalResponse += "service start date: " + str(service_start_date.get('content')) + "\n"

        service_end_date = invoice.fields.get("ServiceEndDate")
        if service_end_date:
           finalResponse += "service end date: " + str(service_end_date.get('content')) + "\n"

        service_address = invoice.fields.get("ServiceAddress")
        if service_address:
           finalResponse += "service address: " + str(service_address.get('content')) + "\n"

        service_address_recipient = invoice.fields.get("ServiceAddressRecipient")
        if service_address_recipient:
           finalResponse += "service address recipient: " + str(service_address_recipient.get('content')) + "\n"

        remittance_address = invoice.fields.get("RemittanceAddress")
        if remittance_address:
           finalResponse += "remittance address: " + str(remittance_address.get('content')) + "\n"

        remittance_address_recipient = invoice.fields.get("RemittanceAddressRecipient")
        if remittance_address_recipient:
           finalResponse += "remittance address recipient: " + str(remittance_address_recipient.get('content')) + "\n"

    return finalResponse

### Creating Agent with the Code Interpreter Tool

In [ ]:
agent_name = "invoice-analysis-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful AI Assistant with code interpreter capabilities.",
        tools = [
            CodeInterpreterTool(
                container = CodeInterpreterToolAuto()
            )
        ]
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Chaining it All Together

In [ ]:
def analyze_invoices_in_bulk():
    final_reponse_string = ""

    for invoice_url in [
        "https://raw.githubusercontent.com/Azure/AI-Fundamentals/main/Foundry/Multi-Modal-Apps/Invoice_Extraction/invoice1.png",
        "https://raw.githubusercontent.com/Azure/AI-Fundamentals/main/Foundry/Multi-Modal-Apps/Invoice_Extraction/invoice2.jpg"
    ]:
        # Analyze the invoice to extract relevant information
        extracted_info = analyze_invoice(invoice_url)
        
        # Redact PII from the extracted information
        redacted_info = PII_Text_Redaction(extracted_info)

        final_reponse_string = final_reponse_string + f"Redacted information from invoice at {invoice_url}:\n{redacted_info}\n\n"
    
    return final_reponse_string

final_response = analyze_invoices_in_bulk()

# Calling Our Agent Finally
response = openai_client.responses.create(
    conversation = conversation.id,
    input = """Could you please create a chart with the invoice total on the y-axis and invoice names on the x-axis?
               Following is the redacted information extracted from the invoices:
               
               """ + final_response,
    extra_body = {
        "agent": {
            "name": agent.name,
            "type": "agent_reference"
        }
    }
)

print(f"Response completed with id: {response.id}")

### Extracting File Information from Response Annotations

In [ ]:
file_id = ""
filename = ""
container_id = ""

# Get the last message which should contain file citations
last_message = response.output[-1]  # ResponseOutputMessage
if last_message.type == "message":
        # Get the last content item (contains the file annotations)
        text_content = last_message.content[-1]  # ResponseOutputText
        if text_content.type == "output_text":
            # Get the last annotation (most recent file)
            if text_content.annotations:
                file_citation = text_content.annotations[-1]  # AnnotationContainerFileCitation
                if file_citation.type == "container_file_citation":
                    file_id = file_citation.file_id
                    filename = file_citation.filename
                    container_id = file_citation.container_id
                    print(f"Found generated file: {filename} (ID: {file_id})")

# Download the generated file if available
if file_id and filename:
        file_content = openai_client.containers.files.content.retrieve(file_id=file_id, container_id=container_id)
        with open(filename, "wb") as f:
            f.write(file_content.read())
            print(f"File {filename} downloaded successfully.")
        print(f"File ready for download: {filename}")
else:
        print("No file generated in response")